### Energy Resolution

The goal of this notebook is to compare the achieveable energy resoultion of the optimal filter (gen2) with that of the most successful machine learning model, using the same resonator, readout, and noise parameters. The plan is to have the optimal filter template and filter generated in the same way that would be done during a MEC run, using a dataset consisting of pulses of the same height. The machine learning model will have been trained on a variety of pulses within the same range. The largest pulse height used to train the machine learning model will be used to generate the template/filter in the optimal filter code. The results will then be compared on the single pulse height. The process will be repeated, generating the optimal filter template/filter with one pulse but the input phase timestream will have varied pulse heights using the same range that was used when training the model.

#### Single Pulse Height

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pathlib
from pathlib import Path
import os
import random
import torch

from torch.utils.data import TensorDataset
from mkidreadoutanalysis.quasiparticletimestream import QuasiparticleTimeStream
from mkidreadoutanalysis.resonator import Resonator, RFElectronics, ReadoutPhotonResonator, FrequencyGrid, LineNoise
from mkidreadoutanalysis.optimal_filters.make_filters import Calculator
from mkidreadoutanalysis.mkidnoiseanalysis import apply_lowpass_filter, compute_r
from mkidcore.config import ConfigThing
from mlcore.dataset import load_training_data

from mlcore.models import BranchedConvReg
from mlcore.training import make_predictions
from mlcore.dataset import load_training_data, stream_to_arrival, create_windows
from mlcore.eval import plot_stream_data

First, the calibration data to be used for generating the optimal filter template/filters needs to be imported.
This data is independent of the actual signal data but will also be used as input to the machine learning model so that
the final performance can be compared to the optimal filter. 

In [ ]:
# Define common parameters
EDGE_PAD = 100
NOISE_SCALE = 30
NUM_SAMPLES = 30000
WINDOW = 1000
MAG = 1.000
FS = 1e6
NOISE_ON = True

In [ ]:
# Define data storage parent location
data_parent_dir = os.environ['ML_DATA_DIR']
data_dir = Path(data_parent_dir + '/pulses/test/single_pulse/variable_qp_density/normalized_iq')
cal_p = Path(data_dir, f'vp_single_num{NUM_SAMPLES}_window{WINDOW}_mag{int(MAG * 1000)}_noisescale{NOISE_SCALE}_pad{EDGE_PAD}.npz')

# Import the photon arrival data from the stored calibration data
cal_i, cal_q, cal_ml_phase_resp, cal_photon_arrs = load_training_data(cal_p, labels=('i', 'q', 'phase_response', 'photon_arrivals'))

The goal is to generate a phase response timestream using a stitched together version of all of the calibration samples,
which currently are stored in the form that allows for training the machine learning model. There are 30000 "training samples",
each with a length of 1000 time samples. There is a photon event somewhere within the middle `1000 - 2 * edge_pad` time samples in each training
sample. Since the optimal filter simulator is designed to accept a single phase response stream, the photon arrival times will need to be unwrapped
and then used to create a phase response time stream, using the ***same resonator, readout, and noise objects used in the machine learning model training set***. Alternatively, the phase timestreams stored in the calibration data
could be stitched together in this way, but the noise additions at the window edges would most likely not be physical.

In [ ]:
# Reshape the calibration photon arrival array
cal_photon_arrs = cal_photon_arrs.flatten()

In [ ]:
# Create the ML model matched qpt/resonator/noise/readout objects to be used to create the phase response timestream

qptimestream = QuasiparticleTimeStream(FS, 30) # Need the qpt timestream to have a length equal to the calibration stream
qptimestream.photon_arrivals = cal_photon_arrs # Manually setting the photon arrivals as opposed to having the object generate its own
qptimestream.gen_quasiparticle_pulse(magnitude=0.6)
_ = qptimestream.populate_photons()

# original amplitudes: 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.01
RES = Resonator(f0=4.0012e9, qi=200000, qc=15000, xa=1e-9, a=0, tls_scale=1e2)
FREQ_GRID = FrequencyGrid(fc=RES.f0, points=1000, span=500e6)
LINE_NOISE = LineNoise(freqs=[60, 50e3, 100e3, 250e3, -300e3, 300e3, 500e3],
                        #amplitudes=[0, 0, 0, 0, 0, 0, 0],
                        #amplitudes=[0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.0001],
                        amplitudes=[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.005],
                        #amplitudes=[0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.01],
                        phases=[0, 0.5, 0,1.3,0.5, 0.2, 2.4],
                        n_samples=100,
                        fs=FS)
RF = RFElectronics(gain=(3.0, 0, 0),
                    phase_delay=0,
                    white_noise_scale=30,
                    line_noise=LINE_NOISE,
                    cable_delay=50e-9)
readout = ReadoutPhotonResonator(RES, qptimestream, FREQ_GRID, RF, noise_on=NOISE_ON)

# Generate phase response time stream.
cal_phase_response, _ = readout.basic_coordinate_transformation()

In [ ]:
# Get an idea of the pulse shape of the calibration data
plt.plot(np.arange(cal_phase_response[:1000].size),cal_phase_response[:1000])
np.sum(cal_photon_arrs[cal_photon_arrs == 1]) # Number of pulses in the calibration set

In [ ]:
# Before sending the phase response time stream to the optimal filter step, it needs to be low-pass filtered.
# The following filter coefficients are pulled from the example notebook in the mkidreadoutanalysis package.
# Current 8-Tap Equirippple Lowpass Exported from MATLAB
coe = np.array([-0.08066211966627938,
                0.02032901400427789,
                0.21182262325068868,
                0.38968583545138658,
                0.38968583545138658,
                0.21182262325068868,
                0.02032901400427789,
                -0.08066211966627938])
low_pass_cal_resp = apply_lowpass_filter(coe, cal_phase_response)
plt.plot(np.arange(low_pass_cal_resp[:1000].size), low_pass_cal_resp[:1000])

Now that the phase response timestream has been created based on the photon arrivals in the calibration data, the accompanying optimal filter can be built

In [ ]:
# Import a ConfigThing object to store optimal filter nerd knob values
cfg_thing = ConfigThing()

# Now define/store the values in the ConfigThing object
cfg_thing.registerfromkvlist(
    (
        ('dt', 1/FS), # Sampling interval (in secs)
        ('fit', True), # This flag instructs the code to fit or not to fit the created template of photon pulses. The template is made by averaging all the pulses in the passed in phase stream after median subtraction. (by default)
        ('summary_plot', True), # The summary plot gives information about the template creation, filtering and the fit to the template (if enabled)
        ('pulses.unwrap', False), # Phase unwrapping refers to recovering the true phase of a signal after it has been "wrapped" into an arbitrary range (E.g. from -pi to pi). Discontinuities in the wrapped stream typically indicate the phase was wrapped. 
        ('pulses.fallback_template', 'default'), # This tells the code which fallback template to use in case making a "good" template couldn't be made from the data (typically due to not enough pulses in the passed in stream.)
        ('pulses.ntemplate', 1000), # Used in the pulse averaging function. Essentially the length of the pulse template.
        ('pulses.offset', 20), # Offset from the start of the template "window" where the pulse will start.
        ('pulses.threshold', 8), # Only pulses greater than this value multiplied by std dev. of all the filtered pulses will be included in the output.
        ('pulses.separation', 50), # A pulse arriving in a time window shorter than this value (in us) with respect to the previous pulse will be discarded.
        ('pulses.min_pulses', 10000), # Number of pulses needed to make a "good" template.
        ('noise.nwindow', 1000), # The size of the overlapping segment used to create the PSD of the phase stream using Welch's method. This is similar to the window size in the STFT when creating Spectrograms (it's called Periodogram).
        ('noise.isolation', 100), # When making the noise spectrum for the data using Welch's method, having pulses too close to each other can skew results. This parameter helps determine how close is too close.
        ('noise.max_windows', 2000), # maximum number of windows of length nwindow needed when creating the noise spectrum of the phase stream using Welch's method.
        ('noise.max_noise', 5000), # cant seem to find this value anywhere in the Calculator class
        ('template.percent', 80), # Pulses that lie outside the middle "percent" percentiles are discarded when creating the pulse template. Higher number means more fringe pulses are used when making the template.
        ('template.cutoff', 0.1), # The filter response of the chosen filter is 0 for frequencies above this value (units in 1/dt)
        ('template.min_tau', 5), # Tau seems to be variable that parameterizes the integral of the normalized template. This parameter describes the minimum value that tau can have to signify a "good" template.
        ('template.max_tau', 500), # In the context of the previous parameter, this is the max value of tau to consider a template a "good" template
        ('template.fit', 'triple_exponential'), # If fitting the template, this is the fitting function to use. These are defined in the templates.py file.
        ('filter.filter_type', 'wiener'), # The type of filter to use for the optimal filter. These are defined in the filters.py file.
        ('filter.nfilter', 100), # The number of taps to use in the chosen filter. 
                                # Jenny Note: For messing around this should be closer to 1000 and ntemplate should be increased to be 5-10x nfilter
                                # Jenny Note: Need to make sure filter is periodic and this gets hard when the filter is short
        ('filter.normalize', True) # If true, normalizes the filter to a unit response.
    ),
namespace='' # Not relevant for the optimal filter code.
)

optimal_filter = Calculator(low_pass_cal_resp, config=cfg_thing, name='calibration')
optimal_filter.calculate()
optimal_filter.plot()

With the optimal filter created and the template generated based on the calibration data, need to now get the calibration data through the highest performing ml model.

In [ ]:
# Define saved model path
MODEL_DIR = pathlib.Path().cwd() / 'best_models'
MODEL_FNAME = 'cnn_reg_1692139221.pt'

# Device determination
if torch.cuda.is_available():
  device = torch.device("cuda")

elif torch.backends.mps.is_available():
  device = torch.device('mps')

else:
  device = torch.device("cpu")
print(f'Using device: "{device}"')

# Create model instance and load trained model
model = BranchedConvReg(in_channels=2, h_hidden_units=100, h_hidden_layers=3)
model.load_state_dict(torch.load(MODEL_DIR / MODEL_FNAME, map_location=device))

Transform data such that it can be passed through the model (conversion from np.array to torch.tensor, split the input and target data, make into a Pytorch Dataset)

In [ ]:
I = readout.normalized_iq.real
Q = readout.normalized_iq.imag

To keep from having to make different data files for each change to the noise parameters, will just create
the dataset dynamically using the photon arrival timestream from an existing data file. In addition to the
previously mentioned increase to agility, this will make the results between the ML and OFC more comparable.

In [ ]:
# Reshape the arrays such that they can be used in the ML data transformation
# functions.
I = I.reshape(30000, 1000)
Q = Q.reshape(30000, 1000)
cal_phase_response = cal_phase_response.reshape(30000, 1000)
cal_photon_arrs = cal_photon_arrs.reshape(30000, 1000)

In [ ]:
# Now we want to expand the dimensions for the i and q streams
# since they will be used as input samples.
cal_i = np.expand_dims(I, axis=1)
cal_q = np.expand_dims(Q, axis=1)

# Get pulse heights and photon arrival values
target_arrs_cal = stream_to_arrival(cal_photon_arrs)
target_pulse_cal = np.min(cal_phase_response, axis=1, keepdims=True)

# Now we want to convert the loaded data to tensors.
# Shape for targets is NUM_SAMPLES x 1 x 2
# Shape for inputs is NUM_SAMPLES x 2 x WINDOW_SIZE
X_cal = torch.Tensor(np.hstack((cal_i, cal_q)))
y_cal = torch.Tensor(np.stack((target_arrs_cal, target_pulse_cal), axis=2))

# From the newly created tensors, create testing and training datasets
cal_dataset = TensorDataset(X_cal, y_cal)

Now that the Pytorch dataset has been created from the calibration data, need to get the model predictions on the calibraiton data

In [ ]:
# Pick k random samples/labels from the test data and plot them along with the predictions
cal_samples = []
cal_labels = []

for sample, label in random.sample(list(cal_dataset), k=len(cal_dataset)): # random.sample randomly samples k elements from the given population without replacement; returns list of samples.
    cal_samples.append(sample)
    cal_labels.append(label)

print(f'Sample Shape: {cal_samples[0].shape}, Label Shape: {cal_labels[0].shape}')
preds = make_predictions(model, [x.unsqueeze(dim=0) for x in cal_samples], device=device) # returns a list
print(f'Preds shape {preds[0].shape}')

With the model predictions on the calibration dataset know, the histograms of both the optimal filter phase response values and the models phase response values can be plotted.

In [ ]:
# Get pulses from the optimal filter with the threshold used when creating the template
_ = optimal_filter.apply_filter()
ofc_phase_preds, _ = optimal_filter.compute_responses(threshold=cfg_thing.get('pulses.threshold'))

# Now randomly pick number of samples from the ml predictions based on the number of
# pulses in the ofc phase preds
sampled_ml_preds = [pred for pred in random.sample(preds, k=ofc_phase_preds.size)]

# Create np array of the sampled ml preds
model_phase_preds = np.array([pred[1] for pred in sampled_ml_preds])

# Create the histograms of model and ofc predictions for pulse height
n_bins = 200
ml_counts, ml_bins = np.histogram(model_phase_preds, range=(-2.2, -1), bins=n_bins)
ofc_counts, ofc_bins = np.histogram(ofc_phase_preds, range=(-2.2, -1), bins=n_bins)

# Lets plot the histogram
plt.figure(figsize=(12,7))
plt.stairs(ml_counts, ml_bins, label='ml')
plt.stairs(ofc_counts, ofc_bins, label='ofc')
plt.title('Predicted Pulse Heights')
plt.xlabel('Pulse Height')
plt.ylabel('Counts')
plt.legend()
plt.show()

In [ ]:
# Verifying both are the same size
print(ofc_phase_preds.size)
print(model_phase_preds.size)

Now that we have filtered and raw peaks, we can use the machinery in the mkidreadoutanalysis package to determine R for both the optimal filter output and the model predictions

In [ ]:
compute_r(ofc_phase_preds, plot=True)

In [ ]:
compute_r(model_phase_preds, plot=True)